# Orchestration Frameworks for LLM Applications

This notebook explores orchestration frameworks that simplify the process of designing, composing, and managing large language model (LLM) applications. We'll focus on two popular open-source orchestration frameworks:

- **LangChain**
- **Semantic Kernel (SK)**

Both help developers connect LLMs with tools, memory, APIs, and logic in reusable and modular ways. This notebook includes a brief introduction to orchestration concepts, followed by quickstart examples with each framework.
"""

What is an Orchestration Framework?
"""
An **orchestration framework** in the context of LLMs enables the structured combination of:
- Language models (like OpenAI's GPT-4)
- Tools and APIs
- Memory and state management
- Logic and control flow (e.g., chains, planners, agents)

The goal is to build complex AI workflows that:
- Maintain modularity and readability
- Enable debugging and monitoring
- Support integration with real-world systems

These frameworks abstract boilerplate code, promote reusability, and facilitate best practices.


**Key Differences**:
- LangChain is Python-native and community-driven
- Semantic Kernel is backed by Microsoft, designed to work with C#, Python, and Java

Each has its strengths, and the choice often depends on:
- Preferred programming language
- Tooling and integration needs
- Use case based (Research and prototyping vs scalable production workflows)




### When to use Langchain

- You want to **experiment fast** and build **interactive AI agents**  
- Your application involves **multiple LLMs or data sources**  
- You prefer a **rich ecosystem of tools and integrations**

---

Langchain gives you the **freedom to innovate quickly** — ideal when flexibility and fast iteration are key.

---


In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
# Instantiate Langchain Chat Model with Azure OpenAI

# Langchain wraps the Azure OpenAI client via AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name=os.getenv("deployment"),
    openai_api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)
#Define a Simple Chat Prompt and Get Response

messages = [
    SystemMessage(content="You are an AI assistant that helps people find information."),
    HumanMessage(content="Who were the founders of Microsoft?")
]

response = llm(messages)

print(response.content)


Microsoft was founded by Bill Gates and Paul Allen. They established the company on April 4, 1975.


In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize memory object
memory = ConversationBufferMemory()

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# First question
print(conversation.predict(input="Who were the founders of Microsoft?"))

# Second question — will have context
print(conversation.predict(input="When did they start the company?"))
# Why this matters: With just a few lines, your LLM becomes context-aware — useful for dialogue agents, customer support, etc.



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Who were the founders of Microsoft?
AI:

> Finished chain.
Microsoft was founded by Bill Gates and Paul Allen. The company was established on April 4, 1975. The two childhood friends from Seattle shared a passion for computers and saw the potential for software development in the burgeoning personal computing industry. Their first major success was developing and selling a BASIC interpreter for the Altair 8800, an early personal computer. This success laid the groundwork for the growth of Microsoft into one of the largest software companies in the world, with its flagship product, the Windows operating system, becoming a dominant force in the industry.


> Ente

In [7]:
# Cell 1: Define a custom tool that simulates a function call
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
def get_company_founders(company: str) -> str:
    founders = {
        "Microsoft": "Bill Gates and Paul Allen",
        "Google": "Larry Page and Sergey Brin"
    }
    return founders.get(company, "Unknown")

# Wrap it as a Langchain tool
founder_tool = Tool(
    name="GetFounders",
    func=get_company_founders,
    description="Use this to get the founders of a company. Input should be a company name."
)
# Cell 2: Call via Langchain Agent with the tool
agent = initialize_agent(
    tools=[founder_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.run("Who founded Google?")



> Entering new AgentExecutor chain...
I should use the GetFounders tool to find out who founded Google. 
Action: GetFounders
Action Input: Google
Observation: Larry Page and Sergey Brin
Thought:I now know the final answer.
Final Answer: Google was founded by Larry Page and Sergey Brin.

> Finished chain.


'Google was founded by Larry Page and Sergey Brin.'

In [8]:
# Cell 1: Simulate a weather lookup tool
from langchain.tools import Tool

def get_weather(city: str) -> str:
    weather_data = {
        "Paris": "Sunny, 25°C",
        "London": "Cloudy, 18°C",
        "New York": "Rainy, 20°C"
    }
    return weather_data.get(city, f"Sorry, no weather data available for {city}.")

weather_tool = Tool(
    name="GetWeather",
    func=lambda input_text: get_weather(input_text),
    description="Get the current weather for a city. Input should be a city name."
)
# Cell 2: Use with Langchain Agent
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

response = agent.run("What is the weather like in Paris?")
print(response)



> Entering new AgentExecutor chain...
To answer the question, I need to find the current weather in Paris.

Action: GetWeather
Action Input: Paris
Observation: Sunny, 25°C
Thought:I now know the final answer. 
Final Answer: The current weather in Paris is sunny with a temperature of 25°C.

> Finished chain.
The current weather in Paris is sunny with a temperature of 25°C.


# Semantic Kernel: From Prototypes to Production-Grade AI Workflows 🚀

---

### What is Semantic Kernel?

Semantic Kernel (SK) is an AI orchestration framework designed for **building robust, scalable AI workflows** that go beyond prototyping.

---

### Key Concepts

- **Plugins:** Modular AI components that perform specific tasks or business logic.  
- **Planners:** Orchestrators that combine plugins into multi-step, context-aware workflows.  

---

### Why Semantic Kernel? 🌟

| Feature                     | Semantic Kernel                         | Langchain / Prototyping Tools          |
|-----------------------------|---------------------------------------|---------------------------------------|
| Production Readiness         |  Native RBAC, error handling, logging | Mainly prototyping and experimentation |
| Workflow Orchestration       |  Complex, stateful, multi-step       |  Basic chaining or manual orchestration |
| Enterprise Integration       |  Built-in support for APIs, identity |  Limited native integration          |
| Governance & Compliance      |  Versioning & lifecycle management   |  Requires extra layers                |

---

### When to Use Semantic Kernel

- You need **scalable, secure, and maintainable AI workflows**  
- Your project demands **deep integration with existing business systems**  
- You want **built-in governance and enterprise-grade controls**

---

Semantic Kernel bridges the gap between AI experiments and real-world production applications — empowering teams to deliver **reliable, secure, and intelligent automation** at scale.

---


# Semantic Kernel Plugins: The building blocks of intelligent workflows 

Plugins in Semantic Kernel are **modular AI components** designed to encapsulate specific functionality — whether it's calling an LLM with a custom prompt, integrating with APIs, or executing domain-specific logic.

Unlike Langchain’s more free-form chaining of tools, Semantic Kernel plugins are **first-class citizens** with built-in support for:

- **Native parameterization and templating**
- **Reusable and composable logic**
- **Tighter integration with planners and memory**
- **Enterprise features like RBAC and versioning**

Plugins help SK deliver **scalable, maintainable, and production-ready AI workflows**, allowing teams to build complex systems by composing these modular blocks.

---

### Simple Plugin Example: Menu Plugin
Defines functions to:
List daily specials (e.g., soups, salads, drinks)
Return prices for requested menu items

In [11]:
import os
from typing import Annotated
from pydantic import BaseModel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments

# Define the plugin with semantic kernel functions
class MenuPlugin:
    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"

class MenuItem(BaseModel):
    price: float
    name: str

async def main():
    # Configure structured outputs format
    settings = OpenAIChatPromptExecutionSettings()
    settings.response_format = MenuItem

    # Create the ChatCompletionAgent with the Azure OpenAI service and the plugin
    agent = ChatCompletionAgent(
        service=AzureChatCompletion(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("api_version"),
            deployment_name=os.getenv('deployment'),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
        ),
        name="SK-Assistant",
        instructions="You are a helpful assistant.",
        plugins=[MenuPlugin()],
        arguments=KernelArguments(settings),
    )

    # Ask the agent a question
    response = await agent.get_response("What is the price of the soup special?")
    print(response.content)

await main()


{"price":9.99,"name":"Soup Special: Clam Chowder"}
